# 5. Analisi e Visualizzazione dei Cluster

**Obiettivo:** Interpretare i cluster identificati per definire i diversi stili di gioco. L'analisi si basa sulle caratteristiche statistiche medie di ciascun gruppo.

**Fasi:**
1.  **Caricamento Dati:** Lettura del DataFrame con i risultati del clustering.
2.  **Calcolo dei Profili Medi:** Calcolo delle statistiche medie per ogni cluster per comprenderne le caratteristiche dominanti.
3.  **Visualizzazione Comparativa (Radar Chart):** Creazione di un radar chart per confrontare visivamente i profili multidimensionali dei cluster.
4.  **Identificazione di Giocatori Rappresentativi:** Elenco di giocatori noti per ogni cluster per validare e dare un nome ai profili identificati.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col

# Aggiunge la root del progetto al sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importazione delle utility e delle funzioni di analisi
from src.utils.helpers import get_spark_session
from src.clustering.models import get_cluster_profiles

# Inizializzazione della sessione Spark
spark = get_spark_session(app_name="NBA_Cluster_Analysis")

# Impostazioni di stile per i grafici
sns.set_style("whitegrid")

### Fase 1: Caricamento dei Risultati del Clustering

In [ ]:
# Carica il DataFrame con i cluster assegnati
clustered_path = "../data/processed/players_clustered.parquet"
clustered_df = spark.read.parquet(clustered_path)

# Lista delle feature utilizzate per il clustering
feature_cols = [
    'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min', 
    'stl_per_36_min', 'blk_per_36_min', 'tov_per_36_min',
    'ts_pct_calc'
]

### Fase 2: Analisi dei Profili Medi dei Cluster

Calcoliamo le statistiche medie per ogni cluster. Questo ci permette di creare un "identikit" statistico per ciascun gruppo e di iniziare a interpretarne lo stile di gioco.

In [ ]:
# Utilizza la funzione helper per calcolare i profili medi e converte in Pandas
cluster_profiles_df = get_cluster_profiles(clustered_df, feature_cols)
cluster_profiles_pd = cluster_profiles_df.toPandas()

# Visualizzazione tabellare dei profili medi
print("Profili Statistici Medi per Cluster:")
display(cluster_profiles_pd.set_index('cluster_id'))

### Fase 3: Visualizzazione dei Profili con Radar Chart

Il radar chart è ideale per confrontare più variabili (le nostre feature) tra diverse categorie (i nostri cluster). Normalizziamo i valori per renderli comparabili sulla stessa scala.

In [ ]:
from sklearn.preprocessing import minmax_scale

# Preparazione dei dati per il grafico
labels = [c.replace('_per_36_min', '').replace('_calc', '').upper() for c in feature_cols]
num_vars = len(labels)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1] # Chiude il cerchio

# Normalizzazione dei profili per la visualizzazione (scala 0-1)
profiles_scaled = cluster_profiles_pd.copy()
for col_name in cluster_profiles_pd.columns:
    if col_name.startswith('avg_'):
        profiles_scaled[col_name] = minmax_scale(profiles_scaled[col_name])

# Creazione del radar chart
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))

for i, row in profiles_scaled.iterrows():
    values = row[[f'avg_{col}' for col in feature_cols]].tolist()
    values += values[:1] # Chiude il cerchio
    ax.plot(angles, values, label=f"Cluster {row['cluster_id']}", linewidth=2)
    ax.fill(angles, values, alpha=0.15)

# Configurazione estetica del grafico
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels, size=12)
plt.title('Profili Comparati dei Cluster di Giocatori NBA', size=20, color='navy', y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

# Salvataggio della figura
reports_dir = '../reports/figures'
if not os.path.exists(reports_dir):
    os.makedirs(reports_dir)
plt.savefig(os.path.join(reports_dir, 'cluster_radar_chart.png'))
plt.show()

### Fase 4: Esempi di Giocatori per Cluster

Per dare un volto ai numeri, identifichiamo alcuni dei giocatori più noti all'interno di ciascun cluster. Questo aiuta a validare l'analisi e a rendere i profili più concreti e riconoscibili.

In [ ]:
# Mostra i top 5 giocatori per ogni cluster, ordinati per punti (come proxy di notorietà)
print("Giocatori Rappresentativi per Cluster:")
for i in range(cluster_profiles_pd.shape[0]):
    print(f"\n--- Cluster {i} ---")
    clustered_df.filter(col('cluster_id') == i) \
                .orderBy(col('pts_per_36_min').desc()) \
                .select('player', 'season', 'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min') \
                .show(5, truncate=False)

In [ ]:
# Termina la sessione Spark
spark.stop()